In [1]:
!pip install -U accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 7.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 30.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate-1.11.0


In [10]:
!pip install transformers==5.0.0

In [11]:
import transformers
print(transformers.__version__)

5.0.0


# Ekstrak Deskripsi dan Opini

In [12]:
import pandas as pd
import pandas as pd
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm

data = pd.read_csv("/kaggle/input/data-magang-1/df_posts_topic_modelling_embedding.csv")

model_id = "GoToCompany/gemma2-9b-cpt-sahabatai-v1-instruct"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quant_config
)


def build_prompt(caption):
    return f"""\
Tugas: Ekstrak informasi penting dari caption Instagram makanan/minuman secara singkat dan objektif.

ATURAN:
- DESKRIPSI: Apa yang dibahas (makanan/minuman/tempat). Maksimal 1 kalimat.
- OPINI: Kesan/rasa/pengalaman/opini penulis. Jika tidak ada opini jelas → "Netral".
- Jika tidak ada info sama sekali → "Tidak tersedia".
- JANGAN tambah informasi di luar caption.
- Format HARUS persis:

Deskripsi: [isi]
Opini: [isi]

Caption:
"{caption}"
"""

def extract_info(caption, max_new_tokens=150):
    prompt = build_prompt(caption)
    messages = [{"role": "user", "content": prompt}]
    
    encodings = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    )
    
    input_ids = encodings.input_ids.to(model.device)
    attention_mask = encodings.attention_mask.to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            do_sample=False,  # greedy → paling konsisten untuk ekstraksi
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def parse_llm_output(text):
    desc = "Tidak tersedia"
    opin = "Netral"
    
    if "Deskripsi:" in text:
        after_desc = text.split("Deskripsi:")[-1].strip()
        if "Opini:" in after_desc:
            d_part, o_part = after_desc.split("Opini:", 1)
            desc = d_part.strip()
            opin = o_part.strip()
        else:
            desc = after_desc.strip()
    
    desc = re.sub(r'\s+', ' ', desc).strip()
    opin = re.sub(r'\s+', ' ', opin).strip()
    
    if not desc or len(desc) < 5:
        desc = "Tidak tersedia"
    if not opin or len(opin) < 3:
        opin = "Netral"
    
    return desc, opin


deskripsi_list = []
opini_list = []

for caption in tqdm(data['caption_no_cta'], desc="Ekstraksi dengan LLM"):
    if not caption.strip():
        deskripsi_list.append("Tidak tersedia")
        opini_list.append("Netral")
        continue
    
    raw = extract_info(caption)
    d, o = parse_llm_output(raw)
    deskripsi_list.append(d)
    opini_list.append(o)

# Simpan hasil ke dataframe
data['deskripsi'] = deskripsi_list
data['opini'] = opini_list


# print("\nHasil 10 baris pertama setelah ekstraksi:")
# print(data_clean[['caption', 'clean_caption', 'deskripsi', 'opini']].head(10).to_markdown(index=False))

# Simpan hasil (opsional)
data.to_csv("/kaggle/working/extracted_data_sahabatai(gemma).csv", index=False)

Loading weights:   0%|          | 0/464 [00:00<?, ?it/s]

Ekstraksi dengan LLM: 100%|██████████| 399/399 [2:01:12<00:00, 18.23s/it]  


In [16]:
data[['caption_no_cta','deskripsi','opini']]

,caption_no_cta,deskripsi,opini
0,definisi one stop dining yang selalu rame di b...,Tempat makan di Bandung yang ramai dan menawar...,Makanan Indonesia sangat enak dan memiliki ras...
1,steak series + menu baru ketupat dorokdok madu...,Steak series dan ketupat dorokdok madura di te...,"Enak, daging tender & juicy, kematangan pas, h..."
2,hampers imlek premium start from 80k . setiap ...,Hampers Imlek premium dari Taralle Bakehouse,"Positif (cantik, exclusive, kasta tertinggi)"
3,barucimol kuah keju\n. jajanan traditional cim...,Cimol kuah keju,"Rich, creamy, ngeju, ada kick pedesnya dari ch..."
4,new asian peranakan comfort food . meet lazy j...,Makanan khas Asia Peranakan di restoran Lazy J...,"Enak, estetik, dan nyaman."
...,...,...,...
394,iftar jakarta # 17\n. 1001 arabian night ramad...,Iftar di Jakarta dengan menu khas Timur Tengah...,Netral.
395,iftar jakarta #16 . a five-star iftar celebrat...,Iftar di Jakarta dengan menu spesial dari chef...,Netral.
396,introducing the flavors of szechuan with the n...,Menu makanan khas Szechuan yang disajikan oleh...,Netral
397,toms new menu . discover the new menu . highli...,Menu baru di Tom's.,Netral.


# Embedding

In [1]:
import pandas as pd

data = pd.read_csv("/kaggle/input/deskripsi-opini/df_posts_topic_embed_gemma.csv")

In [2]:
data = data.drop(columns = ['embedding'])

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               399 non-null    int64  
 1   post_id          399 non-null    int64  
 2   shortcode        399 non-null    object 
 3   post_url         399 non-null    object 
 4   username         399 non-null    object 
 5   caption          399 non-null    object 
 6   likes_count      399 non-null    int64  
 7   comments_count   399 non-null    int64  
 8   posted_at        399 non-null    object 
 9   scraped_at       399 non-null    object 
 10  metadata         399 non-null    object 
 11  nama_resto       399 non-null    object 
 12  akun_resto       398 non-null    object 
 13  alamat           399 non-null    object 
 14  nomor_telepon    197 non-null    object 
 15  jam_operasional  93 non-null     object 
 16  caption_bersih   399 non-null    object 
 17  alamat_resolved 

In [3]:
data_merged = data

## Info_Lengkap

In [4]:
data_merged["info_lengkap"] = (
    "Tempat ini dideskripsikan sebagai berikut: "
    + data_merged["deskripsi"].fillna("Tidak tersedia deskripsi.")
    + ". Berdasarkan opini pengunjung, "
    + data_merged["opini"].fillna("tidak terdapat opini khusus.")
    + ". Tempat ini termasuk dalam kategori "
    + data_merged["topics"].fillna("kategori tidak diketahui")
    + ". Tempat kuliner ini beroperasi pada jam "
    + data_merged["jam_operasional"].fillna("tidak diketahui")
    + ". Berlokasi di "
    + data_merged["lokasi"].fillna("lokasi tidak diketahui")
    + ". "
)

In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# pastikan caption tidak NaN
texts = data_merged["info_lengkap"].fillna("").astype(str).tolist()

# generate embedding
embeddings = model.encode(
    texts,
    show_progress_bar=True,
    convert_to_numpy=True
)

print(embeddings.shape)

2026-02-23 02:38:45.154328: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771814325.490559      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771814325.576871      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771814326.419360      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771814326.419404      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771814326.419406      55 computation_placer.cc:177] computation placer alr

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

(399, 384)


In [7]:
data_merged["embedding_info_lengkap"] = list(embeddings)

In [8]:
data_merged

,id,post_id,shortcode,post_url,username,caption,likes_count,comments_count,posted_at,scraped_at,...,ig_is_business,lokasi,alamat_final,jam_buka,jam_tutup,topics,deskripsi,opini,info_lengkap,embedding_info_lengkap
0,372,3808095248699290737,DTZEnPZkvRx,https://www.instagram.com/p/DTZEnPZkvRx/,kokokuliner,definisi one stop dining yang selalu rame di b...,3,158,2026-01-12 08:12:47,2026-01-13 10:52:21.522681,...,False,Bandung,"Jl. Pasirkaliki No.219, Bandung",8 pagi,10 malam,"Comfort Place & Casual Place, Homey Place",Tempat makan di Bandung yang ramai dan menawar...,Makanan Indonesia sangat enak dan memiliki ras...,Tempat ini dideskripsikan sebagai berikut: Tem...,"[0.21550901, -0.010001246, -0.07874027, 0.0956..."
1,373,3806671440314533044,DTUA4GgkoS0,https://www.instagram.com/p/DTUA4GgkoS0/,kokokuliner,steak series + menu baru ketupat dorokdok madu...,3,159,2026-01-10 09:03:55,2026-01-13 10:52:21.564355,...,True,Bandung,"Jl. Aceh No.58, Bandung",7 pagi,9 malam,"Buffet & BBQ, Hotel, Premium Meat",Steak series dan ketupat dorokdok madura di te...,"Enak, daging tender & juicy, kematangan pas, h...",Tempat ini dideskripsikan sebagai berikut: Ste...,"[0.21747723, -0.020191083, -0.26515716, -0.007..."
2,374,3805918227780998036,DTRVnarEhOU,https://www.instagram.com/p/DTRVnarEhOU/,kokokuliner,hampers imlek premium start from 80k 🏮🧧\n.\nse...,3,200,2026-01-09 08:07:32,2026-01-13 10:52:21.576503,...,False,Bandung,"Paskal 219, Jl. Pasir Kaliki No.219, Bandung",NaN,NaN,Comfort Place & Casual Place,Hampers Imlek premium dari Taralle Bakehouse,"Positif (cantik, exclusive, kasta tertinggi)",Tempat ini dideskripsikan sebagai berikut: Ham...,"[0.18014966, 0.051286034, -0.15599716, 0.15437..."
3,375,3805373652981186245,DTPZyz3kv7F,https://www.instagram.com/p/DTPZyz3kv7F/,kokokuliner,baru❗️cimol kuah keju\n.\njajanan traditional ...,3,147,2026-01-08 14:05:30,2026-01-13 10:52:21.589256,...,True,Bandung,"Saparua, Bandung",NaN,NaN,"Comfort Place & Casual Place, Hotel",Cimol kuah keju,"Rich, creamy, ngeju, ada kick pedesnya dari ch...",Tempat ini dideskripsikan sebagai berikut: Cim...,"[0.030099105, 0.001804711, -0.06933732, 0.1430..."
4,378,3799376495723546310,DS6GMr7kgbG,https://www.instagram.com/p/DS6GMr7kgbG/,kokokuliner,new asian peranakan comfort food 😋\n.\nmeet la...,3,46,2025-12-31 07:30:22,2026-01-13 10:52:21.702382,...,True,Bandung,"Jl. Taman Pramuka No.183, Bandung",9 pagi,9 malam,Premium Meat,Makanan khas Asia Peranakan di restoran Lazy J...,"Enak, estetik, dan nyaman.",Tempat ini dideskripsikan sebagai berikut: Mak...,"[0.25544477, -0.0031333813, -0.15506102, 0.018..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,967,3593127266186168285,DHdWjYdzAfd,https://www.instagram.com/p/DHdWjYdzAfd/,kokokuliner,iftar jakarta # 17🌙\n.\n1001 arabian night ram...,3,90,2025-03-21 17:50:21,2026-01-13 10:52:28.240870,...,True,Jakarta Pusat,"The Ritz-Carlton Jakarta, Pacific Place",NaN,NaN,"Buffet & BBQ, Comfort Place & Casual Place, Ja...",Iftar di Jakarta dengan menu khas Timur Tengah...,Netral.,Tempat ini dideskripsikan sebagai berikut: Ift...,"[0.45974964, -0.12928325, -0.112651974, -0.020..."
395,968,3593025881486854621,DHc_gCmzwnd,https://www.instagram.com/p/DHc_gCmzwnd/,kokokuliner,iftar jakarta #16 🌙\n.\na five-star iftar cele...,3,21,2025-03-21 14:40:11,2026-01-13 10:52:28.254307,...,True,Jakarta Selatan,InterContinental Jakarta Pondok Indah Hotel & ...,NaN,NaN,"Buffet & BBQ, Comfort Place & Casual Place",Iftar di Jakarta dengan menu spesial dari chef...,Netral.,Tempat ini dideskripsikan sebagai berikut: Ift...,"[0.4003376, -0.07539088, -0.13997743, 0.053686..."
396,969,3592266188929689344,DHaSxDxzOcA,https://www.instagram.com/p/DHaSxDxzOcA/,kokokuliner,introducing the flavors of szechuan @liulipala...,3,89,2025-03-20 13:20:20,2026-01-13 10:52:28.266896,...,False,Jakarta Pusat,"Ayana Midplaza Jakarta Hotel, Kav 10-11, Jalan...",NaN,NaN,"Family Place, Japanese Food",Menu makanan khas Szechuan yang disa

# Simpan ke Vector Database

In [9]:
def to_numpy_embedding(x):
    if isinstance(x, np.ndarray):
        return x

    if isinstance(x, list):
        return np.array(x, dtype=np.float32)

    if isinstance(x, str):
        x = x.strip()
        if x.startswith("[") and x.endswith("]"):
            return np.fromstring(x[1:-1], sep=" ")
        else:
            return np.fromstring(x, sep=" ")

    return None

In [10]:
data_merged["embedding_info_lengkap"] = (
    data_merged["embedding_info_lengkap"]
    .dropna()
    .apply(to_numpy_embedding)
)

In [11]:
data_merged

,id,post_id,shortcode,post_url,username,caption,likes_count,comments_count,posted_at,scraped_at,...,ig_is_business,lokasi,alamat_final,jam_buka,jam_tutup,topics,deskripsi,opini,info_lengkap,embedding_info_lengkap
0,372,3808095248699290737,DTZEnPZkvRx,https://www.instagram.com/p/DTZEnPZkvRx/,kokokuliner,definisi one stop dining yang selalu rame di b...,3,158,2026-01-12 08:12:47,2026-01-13 10:52:21.522681,...,False,Bandung,"Jl. Pasirkaliki No.219, Bandung",8 pagi,10 malam,"Comfort Place & Casual Place, Homey Place",Tempat makan di Bandung yang ramai dan menawar...,Makanan Indonesia sangat enak dan memiliki ras...,Tempat ini dideskripsikan sebagai berikut: Tem...,"[0.21550901, -0.010001246, -0.07874027, 0.0956..."
1,373,3806671440314533044,DTUA4GgkoS0,https://www.instagram.com/p/DTUA4GgkoS0/,kokokuliner,steak series + menu baru ketupat dorokdok madu...,3,159,2026-01-10 09:03:55,2026-01-13 10:52:21.564355,...,True,Bandung,"Jl. Aceh No.58, Bandung",7 pagi,9 malam,"Buffet & BBQ, Hotel, Premium Meat",Steak series dan ketupat dorokdok madura di te...,"Enak, daging tender & juicy, kematangan pas, h...",Tempat ini dideskripsikan sebagai berikut: Ste...,"[0.21747723, -0.020191083, -0.26515716, -0.007..."
2,374,3805918227780998036,DTRVnarEhOU,https://www.instagram.com/p/DTRVnarEhOU/,kokokuliner,hampers imlek premium start from 80k 🏮🧧\n.\nse...,3,200,2026-01-09 08:07:32,2026-01-13 10:52:21.576503,...,False,Bandung,"Paskal 219, Jl. Pasir Kaliki No.219, Bandung",NaN,NaN,Comfort Place & Casual Place,Hampers Imlek premium dari Taralle Bakehouse,"Positif (cantik, exclusive, kasta tertinggi)",Tempat ini dideskripsikan sebagai berikut: Ham...,"[0.18014966, 0.051286034, -0.15599716, 0.15437..."
3,375,3805373652981186245,DTPZyz3kv7F,https://www.instagram.com/p/DTPZyz3kv7F/,kokokuliner,baru❗️cimol kuah keju\n.\njajanan traditional ...,3,147,2026-01-08 14:05:30,2026-01-13 10:52:21.589256,...,True,Bandung,"Saparua, Bandung",NaN,NaN,"Comfort Place & Casual Place, Hotel",Cimol kuah keju,"Rich, creamy, ngeju, ada kick pedesnya dari ch...",Tempat ini dideskripsikan sebagai berikut: Cim...,"[0.030099105, 0.001804711, -0.06933732, 0.1430..."
4,378,3799376495723546310,DS6GMr7kgbG,https://www.instagram.com/p/DS6GMr7kgbG/,kokokuliner,new asian peranakan comfort food 😋\n.\nmeet la...,3,46,2025-12-31 07:30:22,2026-01-13 10:52:21.702382,...,True,Bandung,"Jl. Taman Pramuka No.183, Bandung",9 pagi,9 malam,Premium Meat,Makanan khas Asia Peranakan di restoran Lazy J...,"Enak, estetik, dan nyaman.",Tempat ini dideskripsikan sebagai berikut: Mak...,"[0.25544477, -0.0031333813, -0.15506102, 0.018..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,967,3593127266186168285,DHdWjYdzAfd,https://www.instagram.com/p/DHdWjYdzAfd/,kokokuliner,iftar jakarta # 17🌙\n.\n1001 arabian night ram...,3,90,2025-03-21 17:50:21,2026-01-13 10:52:28.240870,...,True,Jakarta Pusat,"The Ritz-Carlton Jakarta, Pacific Place",NaN,NaN,"Buffet & BBQ, Comfort Place & Casual Place, Ja...",Iftar di Jakarta dengan menu khas Timur Tengah...,Netral.,Tempat ini dideskripsikan sebagai berikut: Ift...,"[0.45974964, -0.12928325, -0.112651974, -0.020..."
395,968,3593025881486854621,DHc_gCmzwnd,https://www.instagram.com/p/DHc_gCmzwnd/,kokokuliner,iftar jakarta #16 🌙\n.\na five-star iftar cele...,3,21,2025-03-21 14:40:11,2026-01-13 10:52:28.254307,...,True,Jakarta Selatan,InterContinental Jakarta Pondok Indah Hotel & ...,NaN,NaN,"Buffet & BBQ, Comfort Place & Casual Place",Iftar di Jakarta dengan menu spesial dari chef...,Netral.,Tempat ini dideskripsikan sebagai berikut: Ift...,"[0.4003376, -0.07539088, -0.13997743, 0.053686..."
396,969,3592266188929689344,DHaSxDxzOcA,https://www.instagram.com/p/DHaSxDxzOcA/,kokokuliner,introducing the flavors of szechuan @liulipala...,3,89,2025-03-20 13:20:20,2026-01-13 10:52:28.266896,...,False,Jakarta Pusat,"Ayana Midplaza Jakarta Hotel, Kav 10-11, Jalan...",NaN,NaN,"Family Place, Japanese Food",Menu makanan khas Szechuan yang disa

In [12]:
!pip install chromadb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 84.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 97.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

## Info_Lengkap_

In [13]:
import chromadb
import ast
import numpy as np

client = chromadb.PersistentClient(
    path="./chroma_kuliner"
)

collection_info = client.get_or_create_collection(
    name="info_lengkap"
)

data_merged = data_merged[data_merged["embedding_info_lengkap"].notna()].reset_index(drop=True)

BATCH_SIZE = 50  # aman di laptop

for start in range(0, len(data_merged), BATCH_SIZE):
    end = start + BATCH_SIZE
    batch = data_merged.iloc[start:end]

    documents = batch["info_lengkap"].astype(str).tolist()
    embeddings = batch["embedding_info_lengkap"].tolist()

    metadatas = [
        {
            "id": int(row.id),
            "username": row.username,
            "nama_resto": row.nama_resto,
            "akun_resto": row.akun_resto_norm,
            "jam_operasional": row.jam_operasional,
            "lokasi": row.lokasi,
            "alamat_final": row.alamat_final,
            "topics": row.topics
            # ❌ info_lengkap JANGAN di metadata
        }
        for row in batch.itertuples()
    ]

    ids = [f"kuliner_{row.id}" for row in batch.itertuples()]

    collection_info.add(
        documents=documents,
        embeddings=embeddings,
        metadatas=metadatas,
        ids=ids
    )

    print(f"Inserted {start} - {end}")

Inserted 0 - 50
Inserted 50 - 100
Inserted 100 - 150
Inserted 150 - 200
Inserted 200 - 250
Inserted 250 - 300
Inserted 300 - 350
Inserted 350 - 400


In [14]:
results = collection_info.get(
    limit=5,
    include=["documents", "metadatas", "embeddings"]
)

for i in range(len(results["ids"])):
    print("="*80)
    print("ID:", results["ids"][i])
    print("Caption:", results["documents"][i])
    print("Embedding length:", len(results["embeddings"][i]))  # jangan print full
    print("Metadata:", results["metadatas"][i])

ID: kuliner_372
Caption: Tempat ini dideskripsikan sebagai berikut: Tempat makan di Bandung yang ramai dan menawarkan berbagai menu, termasuk makanan Indonesia, Western, roti, dan tissue bread.. Berdasarkan opini pengunjung, Makanan Indonesia sangat enak dan memiliki rasa yang kuat. Tempat ini nyaman dan cocok untuk makan siang bersama rekan kerja atau keluarga.. Tempat ini termasuk dalam kategori Comfort Place & Casual Place, Homey Place. Tempat kuliner ini beroperasi pada jam 8 pagi hingga 10 malam. Berlokasi di Bandung. 
Embedding length: 384
Metadata: {'topics': 'Comfort Place & Casual Place, Homey Place', 'jam_operasional': '8 pagi hingga 10 malam', 'alamat_final': 'Jl. Pasirkaliki No.219, Bandung', 'lokasi': 'Bandung', 'akun_resto': 'paskal219bandung', 'username': 'kokokuliner', 'nama_resto': 'PASKAL 219', 'id': 372}
ID: kuliner_373
Caption: Tempat ini dideskripsikan sebagai berikut: Steak series dan ketupat dorokdok madura di tempat makan.. Berdasarkan opini pengunjung, Enak, da

# Retrieval

In [ ]:
from sentence_transformers import SentenceTransformer

user_prompt = "Tempat makan mana saja yang cocok untuk makan bareng keluarga?"  # contoh input user

model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

query_embedding = model.encode(
    user_prompt,
    convert_to_numpy=True
).tolist()

results = collection_info.query(
    query_embeddings=[query_embedding],
    n_results=5,
    include=["documents", "metadatas"]
)

# Augmented

In [15]:
contexts = []

for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
    tambahan_meta = []

    if meta.get("nama_resto"):
        tambahan_meta.append(f"Nama tempat: {meta['nama_resto']}")

    if meta.get("akun_resto"):
        tambahan_meta.append(f"Akun Instagram: @{meta['akun_resto']}")

    if meta.get("alamat_final"):
        tambahan_meta.append(f"Alamat: {meta['alamat_final']}")

    meta_text = "\n".join(tambahan_meta)

    context = f"""
{doc}
{meta_text}
""".strip()

    contexts.append(context)

context_text = "\n\n---\n\n".join(contexts)

final_prompt = f"""
Kamu adalah sistem yang menjawab pertanyaan berbasis informasi tambahan yang saya berikan.

ATURAN WAJIB:
- Jawaban HARUS 100% dalam Bahasa Indonesia
- DILARANG menggunakan bahasa selain Bahasa Indonesia
- Jika tidak ada informasi yang relevan, jawab: "Informasi tidak tersedia dalam data"
- Jawaban HARUS berdasarkan INFORMASI yang diberikan
- DILARANG menambah tempat kuliner baru
- Abaikan informasi yang lokasinya tidak sesuai dengan pertanyaan
- Jawaban harus ringkas, jelas, dan informatif

INFORMASI:
{context_text}

PERTANYAAN:
{user_prompt}
"""

print(final_prompt)


Kamu adalah sistem yang menjawab pertanyaan berbasis informasi tambahan yang saya berikan.

ATURAN WAJIB:
- Jawaban HARUS 100% dalam Bahasa Indonesia
- DILARANG menggunakan bahasa selain Bahasa Indonesia
- Jika tidak ada informasi yang relevan, jawab: "Informasi tidak tersedia dalam data"
- Jawaban HARUS berdasarkan INFORMASI yang diberikan
- DILARANG menambah tempat kuliner baru
- Abaikan informasi yang lokasinya tidak sesuai dengan pertanyaan
- Jawaban harus ringkas, jelas, dan informatif

INFORMASI:
Tempat ini dideskripsikan sebagai berikut: Makan siang di rooftop dengan menu seperti cast iron chicken, katong laksa, soto daging berempah, dan xo fried rice.. Berdasarkan opini pengunjung, Enak, flavorful, dan porsi banyak.. Tempat ini termasuk dalam kategori Comfort Place & Casual Place. Tempat kuliner ini beroperasi pada jam tidak diketahui. Berlokasi di Bandung. 
Nama tempat: DAK
Akun Instagram: @dak.bandung
Alamat: @janevallabandung, Level 9

---

Tempat ini dideskripsikan sebagai

# Generation

In [16]:
!pip install openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
import os

os.environ["NVIDIA_API_KEY"] = "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"

from openai import OpenAI
import os

client = OpenAI(
    api_key=os.environ["NVIDIA_API_KEY"],
    base_url="https://integrate.api.nvidia.com/v1"
)

response = client.chat.completions.create(
    model="qwen/qwen2.5-coder-32b-instruct",
    messages=[
        {
            "role": "system",
            "content": (
                "Kamu adalah sistem Question Answering berbasis retrieval.\n"
                "Jawaban HARUS dalam Bahasa Indonesia.\n"
                "Jawaban hanya berdasarkan informasi.\n"
                "Jangan menambah informasi baru."
            )
        },
        {
            "role": "user",
            "content": final_prompt
        }
    ],
    temperature=0.3,
    max_tokens=300
)

print(response.choices[0].message.content)

Tempat makan yang cocok untuk makan bareng keluarga berdasarkan informasi yang diberikan adalah:

1. TIAN JING LOU - Termasuk dalam kategori Family Place dan menawarkan menu keluarga.
2. WARUNG LODEH - Termasuk dalam kategori Family Place dan dikenal dengan makanan yang enak dan ngenyangin.
3. JURU SAMBAL - Dijelaskan sebagai tempat baru yang luas dan cocok untuk makan bareng keluarga, dengan lebih dari 15 varian sambal nusantara.
